# Select protfolio-id for trades analysis

In [ ]:
portfolio_id = "83015fa3-7459-47cc-af51-9d90a1343697"
end_date = '2021-11-22'

#### imports

In [ ]:
import asyncio
import json
import math
import sys
import datetime
from datetime import date, datetime, timedelta

import iso8601
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics.analysis import (
    calc_batch_revenue,
    count_trades,
    load_trades_by_portfolio,
    trades_analysis,
    symbol_trade_analytics,
)

from liualgotrader.common import config
from liualgotrader.common.types import DataConnectorType
from pandas import DataFrame as df
from pytz import timezone
from talib import BBANDS, MACD, RSI
from liualgotrader.common.data_loader import DataLoader 

%matplotlib inline

nest_asyncio.apply()

In [ ]:
config.data_connector = DataConnectorType.alpaca
end_date = datetime.strptime(end_date, "%Y-%m-%d")
local = pytz.timezone("UTC")
end_date = local.localize(end_date, is_dst=None)

#### Load batch data

In [ ]:
trades = load_trades_by_portfolio(portfolio_id)

In [ ]:
trades

In [ ]:
if trades.empty:
    assert False, "Empty batch. halting execution."

## Display trades in details

In [ ]:
minute_history = DataLoader(connector=DataConnectorType.alpaca)
nyc = timezone("America/New_York")
for symbol in trades.symbol.unique():
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["client_time"].min()
    start_date = start_date.replace(hour=9, minute=30, second=0, tzinfo=nyc)
    end_date =  end_date.replace(hour=16, minute=0, second=0, tzinfo=nyc)
    cool_down_date = start_date + timedelta(minutes=5)
    try:
        symbol_data = minute_history[symbol][start_date:end_date]
    except Exception:
        print(f"failed loading {symbol} from {start_date} to {end_date}  -> skipping!")
        continue
        
    minute_history_index = symbol_data.close.index.get_loc(
        start_date, method="nearest"
    )
    end_index = symbol_data.close.index.get_loc(
        end_date, method="nearest"
    )
    cool_minute_history_index = symbol_data["close"].index.get_loc(
        cool_down_date, method="nearest"
    )
    open_price = symbol_data.close[cool_minute_history_index]
    plt.plot(
        symbol_data.close[minute_history_index:end_index].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    d, profit = symbol_trade_analytics(symbol_df, plt)
    
    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()